In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def f(x):
    return 3*x**2 - 4*x + 5

In [3]:
f(3.0)

20.0

In [4]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
ys

array([100.    ,  91.6875,  83.75  ,  76.1875,  69.    ,  62.1875,
        55.75  ,  49.6875,  44.    ,  38.6875,  33.75  ,  29.1875,
        25.    ,  21.1875,  17.75  ,  14.6875,  12.    ,   9.6875,
         7.75  ,   6.1875,   5.    ,   4.1875,   3.75  ,   3.6875,
         4.    ,   4.6875,   5.75  ,   7.1875,   9.    ,  11.1875,
        13.75  ,  16.6875,  20.    ,  23.6875,  27.75  ,  32.1875,
        37.    ,  42.1875,  47.75  ,  53.6875])

In [5]:
plt.plot(xs, ys)

In [6]:
h = 0.0000001
x = 3
f(x+h)

20.00000140000003

In [7]:
h = 0.0000001  #small increment
#slope = (y(added increment) - y(initial))/ amt of increment in x
(f(x + h) - f(x))/h

14.000000305713911

In [8]:
h = 0.00000001
x = -3
(f(x+h) - f(x))/h

-22.00000039920269

In [9]:
h = 0.00000001
x = 2/3
(f(x+h) - f(x))/h

0.0

In [10]:
# lets get more complex
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
print(d)

4.0


In [11]:
h = 0.0001

#inputs
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
#a += h
#b+=h
c+=h
d2 = a*b +c
print("d1: ",d1)
print("d2: ",d2)
print('slope: ',( (d2) - (d1) )/h)

d1:  4.0
d2:  4.0001
slope:  0.9999999999976694


In [91]:
# value object
# takes a single scalar value that it wraps and keep tracks off
class Value:
    def __init__(self,data, _children=(),_op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label= label
        
    def __repr__(self):       # wrapper function
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self,other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward =  _backward   #we try to save these fxn not call the fxn bcz these fxn's return none, just want to store the fxn
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self,other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def tanh(self):
        n = self.data
        t = (math.exp(2*n)-1) / (math.exp(2*n) +1)
        out = Value(t,(self,),'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
            
        return out 

    def backward(self):
        # made a graph of forward prop
        topo = []
        visited =set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        # set grad for output
        self.grad = 1.0

        #call _backward() in reverse of topo[]
        for node in reversed(topo):
            node._backward()
        
a = Value(2.0, label = 'a')
b = Value(-3.0, label='b')
c = Value(10,label='c')
#a, b
#a + b #add two value objects #if we use this #python will internally call a.__add__(b)
e = a*b; e.label = 'e'
d = e+c; d.label = 'd'
f = Value(-2.0,label='f')
L=d*f
L

Value(data=-8.0)

In [13]:
d._prev

{Value(data=-6.0), Value(data=10)}

In [18]:
def lol():
    h = 0.001

    a = Value(2.0,label='a')
    b= Value(-3.0,label='b')
    c= Value(10.0,label='c')
    e=a*b; e.label='e'
    d=e+c; d.label='d'
    f=Value(-2.0,label='f')
    L=d*f; L.label='L'
    L1=L.data

    a = Value(2.0,label='a')
    b= Value(-3.0,label='b')
    c= Value(10.0,label='c')
    e=a*b; e.label='e'
    d=e+c; d.label='d'
    f=Value(-2.0 + h,label='f')
    L=d*f; L.label='L'
    L2=L.data

    print((L2-L1)/h)

lol()
    

3.9999999999995595


In [85]:
# input x1,x2
x1 = Value(2.0)
x2 = Value(0.0)
# weights 
w1 = Value(-3.0)
w2 = Value(1.0)
#bias of the neuron
b = Value(6.8813735870195432)
#neuron = x1w1 + x2w2 + b
x1w1 = x1*w1
x2w2 = x2*w2
x1w1x2w2 = x1w1 + x2w2
n = x1w1x2w2 + b
#activation fxn
o = n.tanh()  #exp(2z)-1 / exp(2z)+1

In [86]:
o.backward()

In [88]:
print('Grad: ')
print(o.grad)
print(n.grad)
print(x1w1x2w2.grad)
print(b.grad)
print(x1w1.grad)
print(x2w2.grad)
print(x1.grad)
print(x2.grad)
print('')
print('Data: ')
print(o.data)
print(n.data)
print(x1w1x2w2.data)
print(b.data)
print(x1w1.data)
print(x2w2.data)
print(x1.data)
print(x2.data)

Grad: 
1.0
0.4999999999999999
0.4999999999999999
0.4999999999999999
0.4999999999999999
0.4999999999999999
-1.4999999999999996
0.4999999999999999

Data: 
0.7071067811865476
0.8813735870195432
-6.0
6.881373587019543
-6.0
0.0
2.0
0.0


In [76]:
topo = []
visited =set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(o)
topo

[Value(data=6.881373587019543),
 Value(data=0.0),
 Value(data=1.0),
 Value(data=0.0),
 Value(data=-3.0),
 Value(data=2.0),
 Value(data=-6.0),
 Value(data=-6.0),
 Value(data=0.8813735870195432),
 Value(data=0.7071067811865476)]

In [70]:
o.grad= 1.0
o._backward()

In [71]:
n.grad

0.4999999999999999

In [72]:
x1w1x2w2.grad

0.0

In [73]:
n._backward()
x1w1x2w2.grad

0.4999999999999999

In [74]:
b.grad

0.4999999999999999

In [75]:
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()
x1.grad

-1.4999999999999996

In [41]:
x1.grad = w1.data * x1w1.grad  #chain rule
w1.grad = x1.data * x1w1.grad

In [40]:
x2.grad = w2.data * x2w2.grad
w2.grad = x2.data * x2w2.grad

In [39]:
x1w1.grad = 0.5
x2w2.grad=0.5

In [38]:
x1w1x2w2.grad=0.5  #for (+) same as n.grad,  n = x1w1x2w2 + b
b.grad=0.5

In [37]:
n.grad = 0.5  

In [36]:
o.grad=1.0

In [29]:
# o = tanh()
# do/dn = 1 - o**2
1 - o.data**2  #n.grad   #go up to see how manual backward is going 

0.4999999999999999

In [93]:
# bug
a = Value(3.0)
b = a + a  #2a #grad = 2.0  
#here self and other is exactly the same so we are overriding with this statement
# self.grad = 1.0 * out.grad   #a.grad
# other.grad = 1.0 * out.grad  #a.grad

b.backward()

print(b.data)
print(a.data)
print('')
print(b.grad)
print(a.grad)

# we can do #self.grad += 1.0 * out.grad
            #other.grad += 1.0 * out.grad

6.0
3.0

1.0
2.0
